# Nettoyage des données d'Open Food Fact

### Chargement des données

Le jeux de données d'Open Food Fact est composé de presque **1,5 millions produits alimentaires**. C'est beaucoup plus qu'il n'en faut pour développer notre modèle de prédiction du Nutri-Score. Pour garder des temps de calcul raisonnables sans trop perdre en généralité, on va se limiter à une seule catégorie de produit. 

_note technique_ : une petite manipulation est nécessaire ici pour ne pas prendre en compte les colonnes/caractères dont les données contiennent plusieurs types (exp. à la fois nombre et des mots), et qui peuvent potentiellement ralentir les performances de calcul. Comme ces caractères ne vont pas nous intéresser dans cette analyse, on va simplement ne pas les retenir par la suite.

In [1]:
## Les données sont analysées grâce aux outils de Pandas
import pandas as pd

In [2]:
## nombre de caractères/colonnes du jeu de données
ncols = len(list(pd.read_csv('dataset/en.openfoodfacts.org.products.csv',sep='\t', nrows =1)))
## colonnes au type non constant
mixedTypeCol = [0,12,18,19,20,21,25,26,27,29,50,62,171]

## tableau de données sans les colonnes au type non constant
df = pd.read_csv('dataset/en.openfoodfacts.org.products.csv', sep='\t',
                  usecols =[i for i in range(ncols) if i not in mixedTypeCol])

print('Nb total de produits alimentaires : ' + str(df.shape[0]))
print('Nb total d\'indicateurs produit  : ' + str(ncols))

Nb total de produits alimentaires : 1486047
Nb total d'indicateurs produit  : 182


_Chaque produit est décrit par un ensemble de 182 indicateurs._


### Sélection des produits laitiers

La table ci-dessous affiche les 5 premières catégories avec le plus grand nombre de produits. On choisit pour cette étude de se restreindre à la catégorie des produits laitiers (`Milk and dairy products`). Cette catégorie relativement simple présente une statistique accessible et suffisante de presque 80 mille produits.

*note* : les catégories sont définies par le [Pogramme National de Nutrition Santé](https://www.mangerbouger.fr/PNNS), ou PNNS, qui propose depuis 2001 des recommandations nutritionnelles pour prévenir l’apparition de certaines maladies et promouvoir la santé publique.

In [155]:
print('Catégorie de produit       Nb de produits')

df['pnns_groups_1'].value_counts().head(5) ## nombre de produits par catégorie

Catégorie de produit       Nb de produits


unknown                    880193
Sugary snacks              126083
Milk and dairy products     77153
Fish Meat Eggs              72168
Cereals and potatoes        69403
Name: pnns_groups_1, dtype: int64

In [156]:
## dataframe de produits laitiers
dfMilk = df[df['pnns_groups_1'] == 'Milk and dairy products']

_L'échantillon de produits laitiers contient presque 80 mille produits._

### Sélection des indicateurs pretinents

Pour cette étude, on aura besoin des indicateurs nutritionnels utilisés pour calculer le Nutri-Score, le Nutri-Score lui-même et les catégories de produits.

In [157]:
nutriscoreFeatures = [
    'energy_100g',
    'salt_100g',
    'sugars_100g',
    'proteins_100g',
    'saturated-fat_100g',
    'fruits-vegetables-nuts_100g',
    'fiber_100g'
]

In [158]:
listVar = [
    'pnns_groups_1',
    'pnns_groups_2',
    'nutriscore_grade',
    'nutriscore_score',
] + nutriscoreFeatures

In [159]:
dfMilkTrimmed = dfMilk[listVar]
nbProd = len(dfMilkTrimmed)
dfMilkTrimmed.columns.size

11



_On se restreint à 11 indicateurs pour l'étude du Nutri-Score_.

### Validité des données produit

Les données dîtes non-valides sont dans notre cas des produits dont les valeurs nutritionnelles sont aberrantes, i.e. qui ne font pas sens.

En premier lieu, on supprime les valeurs négatives et supérieures à 100 pour les teneurs d'indicateurs nutritionnels comprises entre 0 et 100 g.

In [160]:
indexDrop1 = pd.Index([])
for elt in nutriscoreFeatures[1:]:  ## boucle sur les indicateurs nutritionnels
    masktmp = (dfMilkTrimmed[elt] < 0.)  | (dfMilkTrimmed[elt] > 100.)
    indexDrop1 = indexDrop.union(masktmp[masktmp].index)

dfMilkTrimmed = dfMilkTrimmed.drop(indexDrop) ## enlève du df les valeurs aberrantes
print('Nb de produits aux teneurs en gramme non comprises en 0 et 100g : ' + str(indexDrop1.size))

Nb de produits aux teneurs en gramme non comprises en 0 et 100g : 57


On supprime également d'éventuelles densités d'énergie aux valeurs négatives.

In [161]:
masktmp = dfMilkTrimmed['energy_100g'] < 0.
indexDrop2 = masktmp[masktmp].index
dfMilkTrimmed = dfMilkTrimmed.drop(indexDrop2)
print('Nb de produits aux densités d\'énergie négatives : ' + str(len(dfMilkTrimmed[masktmp])))

Nb de produits aux densités d'énergie négatives : 0


Les produits qui ont un Nutri-Score sans avoir aucune information nutritionnelle ne vont pas nous intéresser.

In [162]:
dfMilkTrimmedNS = dfMilkTrimmed[dfMilkTrimmed['nutriscore_grade'].notna()]
indexDrop3 = dfMilkTrimmedNS[dfMilkTrimmedNS[nutriscoreFeatures].isnull().all(axis='columns')].index
print('Nb de produits avec un Nutri-Score mais sans indicateurs nutritionnels : ' + str(indexDrop3.size))

Nb de produits avec un Nutri-Score mais sans indicateurs nutritionnels : 2


In [163]:
dfMilkTrimmed = dfMilkTrimmed.drop(indexDrop3)

### Nettoyage de produits Nutri-Score mal classés

Certains poduits sont mal classés : le point Nutri-Score ne correspond pas à la bonne lettre Nutri-Score. On abandonne l'étude de ces aliments par la suite.

In [164]:
## association Nutri-Score (Point <----> Lettre)
mapping = {'a' : range(-11,  0),
           'b' : range(  0,  3),
           'c' : range(  3, 11),
           'd' : range( 11, 19),
           'e' : range( 19, 36)}

In [165]:
dfMilkTrimmedNS = dfMilkTrimmed[dfMilkTrimmed['nutriscore_grade'].notna()]
## Indentifie les produits dont la lettre Nutri-Score correspond au point NS
masktmp = dfMilkTrimmedNS.apply(lambda x: x['nutriscore_score'] not in mapping[x['nutriscore_grade']], axis=1)
print('Nb de produits pour lesquelles lettre et point Nutri-Score ne correpondant pas : '
      + str(masktmp.value_counts().loc[True]))

Nb de produits pour lesquelles lettre et point Nutri-Score ne correpondant pas : 4


In [166]:
indexDrop4 = masktmp[masktmp].index
dfMilkTrimmed = dfMilkTrimmed.drop(indexDrop4)

### Sauvegarde du fichier nettoyé

Au final, l'échantillon avec lequel on va travailler est composé de 77 092 produits laitiers pour 11 indicateurs.

In [167]:
dfMilkTrimmed.shape

(77090, 11)

On sauvegarde le tableau de données dans un fichier .csv.

In [168]:
dfMilkTrimmed.to_csv('dataset/en.openfoodfacts.org.dairy-products.csv', index=False)